In [1]:
print("Hello World")

Hello World


### Subject

#### The objective of this mini-project is to train compare different SSL strategies by evaluating them on an Anomaly Detection downstream task. 

You will be using two anomaly detection datasets:

 • The MVTec AD datase (download from MVTec AD). Perform your exper
iments on AT LEAST the following categories: bottle, hazelnut, capsule,
 toothbrush.
 
 • The AutoVI dataset (download from AutoVI). Perform your experiments
 ONLY on the following category: engine wiring.
 
####  For each of the categories, you are asked to:
 
 • Train AT LEAST three different SSL models (masked autoencoder, con
trastive model, inpainting model, colorizing model, siamese network...) using
 only the normal, training data.

• Use the model loss as an anomaly score and evaluate the anomaly score’s
 discriminative power by plotting the ROC curve and the AUROC metric.


#### Points will be awarding according to the following criteria:

• The notebook is clearly written, contains markdown cells with all necessary
 explanations, and the code cells are well commented.

• Explanation and arguments of the different model choices, architecture choices
 and hyper-parameter choices (which need not be the same for all categories!).

• Clear comparison and visualization of the results (plots, tables...)
 Extra bonus points will be awarded according to the following criteria:

• Originality of the model choice (i.e. choosing models different than those
 seen during the TP)

• Theobtained AUROC metrics will be compared between different teams and
 bonus points will be given according to the rankings